<a href="https://colab.research.google.com/github/Knick17/NLP/blob/main/assignment_bert_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Классификация текстов с использованием предобученных языковых моделей.

В данном задании вам предстоит обратиться к задаче классификации текстов и решить ее с использованием предобученной модели BERT.

In [1]:
import json
# do not change the code in the block below
# __________start of block__________
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline
# __________end of block__________

Обратимся к набору данных SST-2. Holdout часть данных (которая понадобится вам для посылки) доступна по ссылке ниже.

In [2]:
# do not change the code in the block below
# __________start of block__________

!wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
# __________end of block__________

--2025-03-26 18:48:02--  https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51581 (50K) [text/plain]
Saving to: ‘texts_holdout.json’

texts_holdout.json  100%[===================>]  50.37K  --.-KB/s    in 0.008s  

2025-03-26 18:48:03 (6.55 MB/s) - ‘texts_holdout.json’ saved [51581/51581]



In [3]:
device = 'cuda'

In [4]:
# do not change the code in the block below
# __________start of block__________
df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)
# __________end of block__________

Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [5]:
!pip install torch transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
import json


In [7]:
# Подгружаем предобученный токенизатор
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Кастомный Dataset
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        # Токенизация текста
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        # Возвращаем словарь с токенами и меткой
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Создаем DataLoader для обучения и тестирования
MAX_LEN = 128
BATCH_SIZE = 16

train_dataset = SentimentDataset(texts_train, y_train, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = SentimentDataset(texts_test, y_test, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
# Загружаем модель BERT для классификации
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Если доступен GPU, то переносим модель на него
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Определяем оптимизатор
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)

# Функция потерь уже встроена в модель BERT, так как она используется для бинарной классификации


In [10]:
# Функция обучения
def train_epoch(model, data_loader, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        # Прогоняем данные через модель
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        logits = outputs.logits

        # Считаем количество правильных предсказаний
        preds = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), torch.mean(torch.tensor(losses))

# Обучение модели
EPOCHS = 3

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, device)
    print(f"Train accuracy: {train_acc:.4f}, Train loss: {train_loss:.4f}")


Epoch 1/3
Train accuracy: 0.8520, Train loss: 0.3425
Epoch 2/3
Train accuracy: 0.9478, Train loss: 0.1487
Epoch 3/3
Train accuracy: 0.9770, Train loss: 0.0705


In [11]:
# Функция для оценки точности модели
def evaluate(model, data_loader, device):
    model = model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    return accuracy_score(y_true, y_pred)

# Оценка модели на тестовой выборке
test_accuracy = evaluate(model, test_loader, device)
print(f"Test accuracy: {test_accuracy:.4f}")


Test accuracy: 0.9115


In [12]:
# Применение модели на holdout-данных
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)

# Преобразуем данные в формат, понятный BERT
holdout_dataset = SentimentDataset(texts_holdout, [0]*len(texts_holdout), tokenizer, MAX_LEN)
holdout_loader = DataLoader(holdout_dataset, batch_size=BATCH_SIZE)

# Прогнозы для holdout
holdout_accuracy = evaluate(model, holdout_loader, device)
print(f"Holdout accuracy: {holdout_accuracy:.4f}")


Holdout accuracy: 0.4600


#### Сдача взадания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к первому (положительному) классу

Несколько `assert`'ов для проверки вашей посылки:

In [13]:
def get_probabilities(model, data_loader, device):
    model = model.eval()
    probabilities = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Прогоняем данные через модель
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Применяем сигмоиду, чтобы получить вероятности для положительного класса
            probs = torch.sigmoid(logits).cpu().numpy()

            # Преобразуем в обычные float, чтобы пройти проверки
            probabilities.extend([float(p) for p in probs[:, 1]])  # Извлекаем вероятность для положительного класса

    return probabilities


In [14]:
# Получение вероятностей для обучающей выборки
train_probs = get_probabilities(model, train_loader, device)

# Получение вероятностей для тестовой выборки
test_probs = get_probabilities(model, test_loader, device)

# Получение вероятностей для данных holdout
holdout_dataset = SentimentDataset(texts_holdout, [0]*len(texts_holdout), tokenizer, MAX_LEN)
holdout_loader = DataLoader(holdout_dataset, batch_size=BATCH_SIZE)
holdout_probs = get_probabilities(model, holdout_loader, device)

# Сохраняем в словарь
out_dict = {
    'train': train_probs,
    'test': test_probs,
    'holdout': holdout_probs
}

# Проверка assert-ов
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert len(out_dict["train"]) == 5000, "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert len(out_dict["test"]) == 1920, "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert len(out_dict["holdout"]) == 500, "The predicted probas list length does not match the holdout set size"


In [15]:
out_dict

{'train': [0.04358886182308197,
  0.1352226287126541,
  0.07286009937524796,
  0.07685784250497818,
  0.9576008915901184,
  0.26222237944602966,
  0.06425642967224121,
  0.9214091897010803,
  0.9123616814613342,
  0.05706194415688515,
  0.07237851619720459,
  0.9598139524459839,
  0.26963719725608826,
  0.049221668392419815,
  0.9568707346916199,
  0.9581943154335022,
  0.9430863857269287,
  0.9543549418449402,
  0.9597111940383911,
  0.13844339549541473,
  0.9366684556007385,
  0.9572373628616333,
  0.9346225261688232,
  0.09931401908397675,
  0.9223546385765076,
  0.9558311104774475,
  0.19975794851779938,
  0.9387591481208801,
  0.9601765275001526,
  0.953779399394989,
  0.9356449246406555,
  0.8909301161766052,
  0.1759052574634552,
  0.9559083580970764,
  0.08369433134794235,
  0.07389358431100845,
  0.04999613016843796,
  0.9292465448379517,
  0.955935537815094,
  0.07007665932178497,
  0.1446160227060318,
  0.8552761077880859,
  0.8926778435707092,
  0.06252045184373856,
  0.058

Запустите код ниже для генерации посылки.

In [16]:
# do not change the code in the block below
# __________start of block__________
FILENAME = "submission_dict_hw_text_classification_with_bert.json"

with open(FILENAME, "w") as iofile:
    json.dump(out_dict, iofile)
print(f"File saved to `{FILENAME}`")
# __________end of block__________

File saved to `submission_dict_hw_text_classification_with_bert.json`


На этом задание завершено. Поздравляем!